In [65]:
import pandas as pd
import requests 
import json

In [66]:
X_train = pd.read_csv('X_train.csv', index_col=0)
X_test = pd.read_csv('X_test.csv',index_col=0)
y_train = pd.read_csv('y_train.csv',index_col=0)
y_test = pd.read_csv('y_test.csv',index_col=0)

In [67]:
# X_train.reset_index(drop=True, inplace=True)
# X_test.reset_index(drop=True, inplace=True)
# y_train.reset_index(drop=True, inplace=True)
# y_test.reset_index(drop=True, inplace=True)

In [68]:
X_train_json = X_train.to_json()
y_train_json = y_train.to_json()
X_test_json = X_test.to_json()
y_test_json = y_test.to_json()

In [69]:
predict = True
timesteps = 10
model_case_version_main_target_code = 5
algorithm_configurations = json.dumps({
                "compile_parameters": {
                    "twoexp_nodes_number_layer_1": 7,
                    "twoexp_nodes_number_layer_2": 10,
                    "twoexp_nodes_number_layer_3": 7,
                    "twoexp_nodes_number_layer_4": 6,
                    "twoexp_nodes_number_layer_5": 0,
                    "attenuated_padding_value": 1,
                    "optimizer": "adam",
                    "iteration": 1
                },
                "fit_parameters": {
                    "batch_size": 64,
                    "n_epochs": 100,
                }
            })
iteration = 1
algorithm_code = 'LTM'

In [70]:
with open('pretreatment_attrs.json', 'r') as json_file:
    pretreatment_attrs = json.load(json_file)

pretreatment_attrs = json.dumps(pretreatment_attrs)

In [71]:
query = f"""
        query {{
                train{algorithm_code} (
                X_train: {json.dumps(X_train_json)}
                y_train: {json.dumps(y_train_json)}
                X_test: {json.dumps(X_test_json)}
                y_test: {json.dumps(y_test_json)}
                predict: {json.dumps(predict)}
                timesteps: {json.dumps(timesteps)}
                model_case_version_main_target_code: {json.dumps(model_case_version_main_target_code)}
                algorithm_configurations: {json.dumps(algorithm_configurations)}
                iteration: {iteration}
                pretreatment_attrs: {json.dumps(pretreatment_attrs)}
            ) {{
                success,
                error,
                main_target_predictions_train,
                main_target_columns
            }}
        }}
    """

In [72]:
headers = {
    "Accept-Encoding": "gzip, deflate, br",
    "Content-Type": "application/json",
    "Accept": "application/json",
    "Connection": "keep-alive",
    "DNT": "1",
}

response = requests.post('http://0.0.0.0:4000/graphql', json={"query": query}, headers=headers)

In [73]:
response = response.json()

In [75]:
main_target_predictions_train = response['data']['trainLTM']['main_target_predictions_train']
main_target_columns = response['data']['trainLTM']['main_target_columns']

post_treated_predictions_df = pd.DataFrame(
    data = main_target_predictions_train,
    columns = main_target_columns
)

In [76]:
post_treated_predictions_df

,MPN5P
0,32.471632
1,32.599488
2,32.449067
3,32.111949
4,31.636899
...,...
5510,29.377283
5511,29.402736
5512,29.380690
5513,29.282247
